In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tqdm
import re
from sklearn import preprocessing
import librosa
from imblearn.combine import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn import svm

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='SimHei', font_scale=1.2, palette=palette, style='white')  # 解决Seaborn中文显示问题

In [4]:
from util import load

df, attachList = load.load(list(range(1, 7 + 1)))

../3rd天府杯A提/附件7\20.txt: 100%|██████████| 140/140 [00:02<00:00, 48.93it/s]


In [5]:
df

,1-1,1-2,1-3,1-4,1-5,1-6,1-7,1-8,1-9,1-10,...,7-11,7-12,7-13,7-14,7-15,7-16,7-17,7-18,7-19,7-20
0,0.031847,0.016663,0.020103,0.005837,-0.011276,-0.025771,0.021692,0.009281,0.010529,0.001247,...,-0.117107,-0.074528,0.624229,0.301207,0.474272,0.045762,-0.668258,1.265104,0.004534,-0.009769
1,-0.011189,-0.002048,0.006537,0.023145,0.012113,-0.017351,0.010933,-0.003817,0.004915,-0.007641,...,-0.113252,-0.086088,0.658908,0.339724,0.458840,-0.077554,-0.833916,1.110840,0.004534,0.005652
2,-0.019609,-0.007661,-0.002818,0.003030,0.021468,-0.009866,0.007658,-0.004752,0.003980,-0.002028,...,-0.113252,-0.105354,0.709001,0.378242,0.466556,-0.154626,-0.972605,0.891012,0.004534,-0.013624
3,0.019685,0.007308,-0.002351,-0.019423,0.004160,-0.003318,0.005787,-0.018318,-0.022216,-0.028223,...,-0.128671,-0.086088,0.766799,0.428314,0.466556,-0.177748,-1.061213,0.651903,0.000677,-0.009769
4,-0.020076,0.001226,-0.020594,0.016596,0.013048,-0.015948,-0.000294,-0.023931,0.003045,-0.024481,...,-0.093978,-0.093794,0.828451,0.470684,0.412548,-0.158480,-1.057360,0.432077,0.000677,0.001797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-2.936200,1.122488,1.303684,-0.291669,-0.145528,0.035976,0.138636,-0.077726,0.077889,-0.176508,...,-2.349002,2.210447,2.928470,1.283404,-2.303295,0.038054,-3.661649,-0.705624,-4.005563,-4.921185
7996,-3.392283,2.000974,0.883152,-0.199985,0.457904,0.205311,0.186349,-0.010833,0.116246,-0.133005,...,1.212779,2.722928,2.905350,1.321920,-2.172132,-0.112237,-3.442057,-0.451088,-3.415616,-4.261961
7997,-2.868840,1.194994,0.248846,0.020338,0.813415,0.481300,0.176526,-0.000075,0.141039,-0.115230,...,3.170988,4.869186,2.893791,1.352735,-2.071831,-0.339600,-3.095332,-0.196552,-2.848805,-3.248065
7998,-2.935732,-0.888954,-0.528132,0.284164,0.943457,0.712382,0.149862,-0.014108,0.181735,-0.082953,...,6.343440,6.714890,2.859112,1.383549,-2.006250,-0.624768,-2.713934,0.131260,-2.393813,-1.937326
